# IMPORTS

In [8]:
import os
from classes.layers.Conv2D import Conv2D
from classes.layers.Dense import Dense
from classes.layers.Flatten import Flatten
from classes.layers.Input import InputLayer
from classes.models.Sequential import Sequential
from classes.utils.ImageConvert import ImageConvert

# DATA PREPROCESSING

In [28]:
ic = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_gen = ic.from_directory(os.path.join('.','data','test'), (256,256), mode='binary', color_mode='rgb')

# MODEL

## Instantiating

In [29]:
model = Sequential([
        InputLayer(input_shape=(256,256,3)),
        Conv2D(2, (16, 16), activation='relu'),
        Flatten(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

model.compile()

## Training

In [ ]:
# NOT YET IMPLEMENTED

## Testing

In [30]:
# CULL DATA
data_gen.shuffle_data_label = data_gen.shuffle_data_label[:10]

# TEST
model.predict(data_gen)

Step: 1/10
Step: 2/10
Step: 3/10
Step: 4/10
Step: 5/10
Step: 6/10
Step: 7/10
Step: 8/10
Step: 9/10
Step: 10/10


{'results': [array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.]),
  array([1.])],
 'true_labels': [1, 1, 0, 0, 1, 0, 0, 0, 0, 0]}